# Bernoulli Formel

In [1]:
import numpy as np

In [2]:
# Calculating speed from given dynamic pressure using Bernoulli equation

def calc_speed_from_pressure(dynamic_pressure, static_pres, temperature):

    # Calculating air density by using the gas constant for dry air (=287.058)  
    density = static_pres / (287.058 * (temperature + 273.15))
    # Applying Bernoulli 
    speed = np.sqrt(2 * (dynamic_pressure/ density))
    # Converting results from m/s to km/h
    speed *= 3.6

    return speed  

In [3]:
# Calculating dynamic pressure from given speed using Bernoulli equation

def calc_pressure_from_speed(speed, static_pres, temperature):

    # Converting speed form km/h to m/s
    speed /= 3.6
    # Calculating air density by using the gas constant for dry air (=287.058) 
    density = static_pres / (287.058 * temperature)
    # Applying Bernoulli 
    dynamic_pressure = (density / 2) * np.power(speed, 2)

    return dynamic_pressure 

# Anwendung der Ordinary Least Square Methode 

In [12]:
import numpy as np
from pathlib import Path


import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('seaborn')

In [2]:
# Loading data from a CSV file.

def load_sensor_data(fname, num_of_col, num_data_points):
    
    data = np.genfromtxt(fname, dtype= int, skip_header=1, delimiter=",", usecols=(num_of_col+2), max_rows=num_data_points) 
    
    return data

In [3]:
''' Generating fitting label vector for data matrix X
        Input:
            num_data_points:  number of data point per measurenment file
            y_labels_90:      points of measurenment 
        Output:
            Y:                1xN array containing "true" labels for data matrix X
'''

def calc_label_vector(num_data_points, y_labels_90):
    
    Y = []
    
    for i in y_labels_90[0,:]: 
        for j in range(num_data_points):
            Y.append(i)

    Y = np.array(Y)

    return Y.T

In [4]:
''' Calculating X-Matrix for: 
        Input: 
            filename:           Path to folder containing data files
            num_sensor:         specifies sensor
            order:              order of calibration function 
            num_data_points:    Points of measurenment
        Output: 
            x:                  Nx(order+1) array --> [x, x^2, x^3, ..., x^n, 1]   
'''

def calc_x_matrix(filename, num_sensor, order, num_data_points):

    X = []
    # loop through files in folder 
    pathlist = Path(filename).glob('**/*.csv')
    i = 0
    for path in sorted(pathlist):
        i += 1
        path_in_str = str(path)

        if i == 1: 
            X = load_sensor_data(path_in_str, num_sensor, num_data_points)
        else: 
            data = load_sensor_data(path_in_str, num_sensor, num_data_points)
            X = np.append(X, data)

        X = X.T

    if order > 1:
        X = calc_X_higher_n(X, order, len(X))
    
    X = np.c_[X, np.ones(X.shape[0], dtype= int)]
    
    return X

In [5]:
''' Calculating columns for order X^2 to X^n.
    Input: 
        X:          NX(n+1) with N data points for a polynomial function of order n --> [X, 1, ..., 1]
        n:          Order of the calibration function
        size:       Amount of data points in matriX X
    Output: 
        X:          NX(n+1) with N data points for a polynomial function of order n --> [X, X^2, X^3, ..., X^n, 1]  
'''

def calc_X_higher_n(X, n, size):

    X_matriX = X
    i = 2
    while i <= n: 
        
        temp = np.power(X, i)
        X_matriX = np.c_[X_matriX, temp]
        i += 1

    return X_matriX

In [6]:
''' Calculating weight vector W containing the values of the calibration function. 
    Input: 
        X:              (n+1)xN array of N data points for a calibration function of order n    
        y_labels:       Physical values/ labels for data points
        num_sensor:     Specifies sensor
    Output: 
        W:              (n+1)x1 weight vector containing calibration parameters 
'''

def calculating_w_ols(X, y_labels):
    
    W = np.dot(np.linalg.inv(np.dot(X, np.transpose(X))), np.dot(X, np.transpose(y_labels)))
    
    return W

In [7]:
''' Applying ordinary least squares (ols) regression
        Input:
            X:      (n+1)xN array of N data points for a calibration function of order n
            W:      (n+1)x1 weight vector containing calibration parameters 
        Output:
            Y:      1xN array containing estimated values
'''

def calc_OLS(W, X):
    
    Y = np.dot(np.transpose(W), X)

    return Y

In [ ]:
'''
    Calculating calibration function and estimated values from given data points for sensor 1 and 90 SPS. 
    Input: 
        filename_90:    Name of folder that holds measurement files - Adjust!
        data_points:    Amount of data points per measurnment file for 90 SPS - Adjust!
'''

filename_90 = '/Users/leaschwalb/Documents/UNI/1_Bachelorarbeit/Messungen/DLR_neu/90SPS'
filename_90_small = '/Users/leaschwalb/Documents/UNI/1_Bachelorarbeit/Messungen/DLR_neu/90small'
data_points = 410

# Class labels in vector Y in Pascal
y_labels_90 = np.array([[-6000, -5000, -4000, -3000, -2000, -1000, -900, -800, -700, -600, -500, -400, -300, -200, -100, 0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 2000, 3000, 4000, 5000, 6000]])
Y = calc_label_vector(data_points, y_labels_90)

# Small: -/+ 1000
Y_small = calc_label_vector(data_points, y_labels_90[:,5:26])

# X & W & Y for linear function for Sensor 1
X_90_lin_1 = calc_x_matrix(filename_90, 1, 1, data_points)
W_90_lin_1 = calculating_w_ols(np.transpose(X_90_lin_1), Y)
y_90_lin_1 = calc_OLS(W_90_lin_1, np.transpose(X_90_lin_1))
print('W90_lin Sensor 1: ' + str(W_90_lin_1))

# X & W & Y for linear function for Sensor 2
X_90_lin_2 = calc_x_matrix(filename_90_small, 2, 1, data_points)
W_90_lin_2 = calculating_w_ols(np.transpose(X_90_lin_2), Y_small)
y_90_lin_2 = calc_OLS(W_90_lin_2, np.transpose(X_90_lin_2))
print('W90_lin Sensor 2 (small): ' + str(W_90_lin_2))

# X & W & Y for linear function for Sensor 3
X_90_lin_3 = calc_x_matrix(filename_90_small, 3, 1, data_points)
W_90_lin_3 = calculating_w_ols(np.transpose(X_90_lin_3), Y_small)
y_90_lin_3 = calc_OLS(W_90_lin_3, np.transpose(X_90_lin_3))
print('W90_lin Sensor 3 (small): ' + str(W_90_lin_3))

# X & W & Y for 2nd order polynomial
X_90_poly2 = calc_x_matrix(filename_90, 1, 2, data_points)
W_90_poly2 = calculating_w_ols(np.transpose(X_90_poly2), Y)
y_90_poly2 = calc_OLS(W_90_poly2, np.transpose(X_90_poly2))
print('W90_poly2: ' + str(W_90_poly2))

# X & W & Y for 3nd order polynomial
X_90_poly3 = calc_x_matrix(filename_90, 1, 3, data_points)
W_90_poly3 = calculating_w_ols(np.transpose(X_90_poly3), Y)
y_90_poly3 = calc_OLS(W_90_poly3, np.transpose(X_90_poly3))
print('W90_poly3: ' + str(W_90_poly3))
 
# X & W & Y for 4nd order polynomial
X_90_poly4 = calc_x_matrix(filename_90, 1, 4, data_points)
W_90_poly4 = calculating_w_ols(np.transpose(X_90_poly4), Y)
y_90_poly4 = calc_OLS(W_90_poly4, np.transpose(X_90_poly4))
print('W90_poly4: ' + str(W_90_poly4))

# X & W & Y for linear function in smaller interval <= 1000Pa
X_90_lin_supersm = calc_x_matrix(filename_90_small, 1, 1, data_points)
W_90_lin_supersm = calculating_w_ols(np.transpose(X_90_lin_supersm), Y_small)
y_90_lin_supersm = calc_OLS(W_90_lin_supersm, np.transpose(X_90_lin_supersm))
print('W90_lin_sm: ' + str(W_90_lin_supersm))

# X & W & Y for polynomial function of 2nd order in smaller interval <= 1000Pa
X_90_poly2_supersm = calc_x_matrix(filename_90_small, 1, 2, data_points)
W_90_poly2_supersm = calculating_w_ols(np.transpose(X_90_poly2_supersm), Y_small)
y_90_poly2_supersm = calc_OLS(W_90_poly2_supersm, np.transpose(X_90_poly2_supersm))
print('W90_poly2_sm: ' + str(W_90_poly2_supersm))

points = np.arange(-12000, 18000, 500)

one_plot_all_functions(points, W_90_lin_1, W_90_poly2, W_90_poly3, W_90_poly4)

In [9]:
'''
    Calculating calibration function and estimated values from given data points for sensor 1, comparing different SPS. 
    Input: 
        filename_90:    Name of folder that holds measurement files - Adjust!
        data_points:    Amount of data points per measurnment file for 90 SPS - Adjust!
'''

filename_20 = '/Users/leaschwalb/Documents/UNI/1_Bachelorarbeit/Messungen/DLR_neu/20SPS'
filename_90_compare = '/Users/leaschwalb/Documents/UNI/1_Bachelorarbeit/Messungen/DLR_neu/90comp'
filename_175 = '/Users/leaschwalb/Documents/UNI/1_Bachelorarbeit/Messungen/DLR_neu/175SPS'
filename_330 = '/Users/leaschwalb/Documents/UNI/1_Bachelorarbeit/Messungen/DLR_neu/330SPS'
filename_1000 = '/Users/leaschwalb/Documents/UNI/1_Bachelorarbeit/Messungen/DLR_neu/1000SPS'

data_points_20 = 99
data_points_90 = 410
data_points_175 = 830 
data_points_330 = 1250
data_points_1000 = 2500

# Class labels in vector Y in Pascal
y_labels_all = np.array([[-6000, -5000, -4000, -3000, -2000, -1000, -500, 0, 500, 1000, 2000, 3000, 4000, 5000, 6000]])
Y_20 = calc_label_vector(data_points_20, y_labels_all)
Y_90 = calc_label_vector(data_points_90, y_labels_all)
Y_175 = calc_label_vector(data_points_175, y_labels_all)
Y_330 = calc_label_vector(data_points_330, y_labels_all)
Y_1000 = calc_label_vector(data_points_1000, y_labels_all)

# X & W & Y for 20 SPS
X_20_1 = calc_x_matrix(filename_20, 1, 1, data_points_20)
W_20_1 = calculating_w_ols(np.transpose(X_20_1), Y_20)
y_20_1 = calc_OLS(W_20_1, np.transpose(X_20_1))
print('W20: ' + str(np.round(W_20_1,4)))

# X & W & Y for 90 SPS
X_90_1 = calc_x_matrix(filename_90_compare, 1, 1, data_points_90)
W_90_1 = calculating_w_ols(np.transpose(X_90_1), Y_90)
y_90_1 = calc_OLS(W_90_1, np.transpose(X_90_1))
print('W90: ' + str(np.round(W_90_1,4)))

# X & W & Y for 175 SPS
X_175_1 = calc_x_matrix(filename_175, 1, 1, data_points_175)
W_175_1 = calculating_w_ols(np.transpose(X_175_1), Y_175)
y_175_1 = calc_OLS(W_175_1, np.transpose(X_175_1))
print('W175: ' + str(np.round(W_175_1,4)))

# X & W & Y for 330 SPS
X_330_1 = calc_x_matrix(filename_330, 1, 1, data_points_330)
W_330_1 = calculating_w_ols(np.transpose(X_330_1), Y_330)
y_330_1 = calc_OLS(W_330_1, np.transpose(X_330_1))
print('W330: ' + str(np.round(W_330_1,4)))

# X & W & Y for 1000 SPS
X_1000_1 = calc_x_matrix(filename_1000, 1, 1, data_points_1000)
W_1000_1 = calculating_w_ols(np.transpose(X_1000_1), Y_1000)
y_1000_1 = calc_OLS(W_1000_1, np.transpose(X_1000_1))
print('W1000: ' + str(np.round(W_1000_1,4)))


W20: [  1.0007 -19.158 ]
W90: [  1.0007 -19.2845]
W175: [  1.0447 -64.0567]
W330: [  1.0007 -18.9817]
W1000: [  1.0008 -18.7376]


In [18]:
''' Plotting linear calibration function against calibration function of nth order (2 <=n<= 4)
    Input: 
        x:          Data points 
        w:          weight vector for linear calibration function
        w_poly:     weight vector for polynomial calibration function of nth order 
'''

def one_plot_all_functions(x, w, w_2, w_3, w_4): 

    y_lin = x * w[0] + w[1]
    y_poly2 = np.power(x,2) * w_2[1] + x * w_2[0] + w_2[2]
    y_poly3 = np.power(x,3) * w_3[2] + np.power(x,2) * w_3[1] + x * w_3[0] + w_3[3]
    y_poly4 = np.power(x,4) * w_4[3] + np.power(x,3) * w_4[2] + np.power(x,2) * w_4[1] + x * w_4[0] + w_4[4]
    
    # Defining style of plots
    sns.set_theme()
    sns.set_context("notebook")
    sns.set_style("whitegrid")

    fig, ax1 = plt.subplots(ncols=1, sharex=True, sharey=True, figsize=(15, 10))
    
    for ax in fig.get_axes():
        ax.set_xlabel('Sensorwerte in Pa', fontweight ='bold', fontsize=12, loc='right')
        ax.set_ylabel('Referenzwerte in Pa', fontweight ='bold', fontsize=12, loc='top')
        ax.label_outer()
        # ax.set(aspect='equal')
        ax.spines['right'].set_color('none')
        ax.spines['top'].set_color('none')
        ax.spines['bottom'].set_position('zero')
        ax.spines['left'].set_position('zero')

    ax1.plot(x, y_lin, linestyle='-', label='Linear', color='lime')
    ax1.plot(x, y_poly2, linestyle='--', label='Polynom 2. Ordnung', color= 'navy')
    ax1.plot(x, y_poly3, linestyle='-.', label='Polynom 3. Ordnung', color= 'green')
    ax1.plot(x, y_poly4, linestyle='-', label='Polynom 4. Ordnung', color= 'dodgerblue')
    ax1.legend(loc='upper left', frameon=True)
    # ax1.set_title("Kalibrierfunktionen", fontsize=15, color="grey")

    # Vertikal red line at position marking the measuring intervall of sensors
    pos = 6000
    plt.plot([pos,pos], [-10000,10000], color ='orangered', linewidth=1.5, linestyle="-.", label='Messbereich der Sensoren')
    plt.plot([-pos,-pos], [-10000,10000], color ='orangered', linewidth=1.5, linestyle="-.")
    
    # Combine all the operations and display
    plt.legend(loc='upper left', frameon=True)
    plt.show()

# Berechnung der Auswertungsparameter 

In [17]:
''' Calculating NRMS (Normalized Root Mean Square Error).
    Input: 
        y_kalib:        Data points after applying calibration function
        y_lables:       Physical values/ labels for data points
        data_points:    Amount of data points
    Output: 
        e:              Root mean square error 
        nrmse:         Normalised root mean square error
'''

def effective_value_of_error(y_kalib, y_labels, data_points):

    rmse = np.sqrt((1 / data_points) * np.sum(np.power((y_labels - y_kalib), 2)))
    nrmse = np.sqrt((np.sum(np.power(((y_labels - y_kalib) / 120), 2))) / data_points) 

    return rmse, nrmse

In [18]:
''' Calculating R^2 Value.
    Input: 
        y_kalib:        Data points after applying calibration function
        y_lables:       Physical values/ labels for data points
        data_points:    Amount of data points
        order:          order of calibration function
    Output: 
        R^2:            Maximum deviation between label and calibrated data point. 
        R^2_korr:       Mean deviation between labels and calibrated data points.
'''

def R_square(y_kalib, y_labels, data_points, order):
    
    R_square = np.sum(np.power((y_kalib - np.mean(y_kalib)), 2)) / np.sum(np.power((y_labels - np.mean(y_labels)), 2))
    R_square_corr = 1 - (((data_points - 1) / (data_points - order - 1)) * (1 - R_square))

    return R_square, R_square_corr

In [19]:
''' Calculating the mean/ maximum deviation.
    Input: 
        y_kalib:    Data points after applying calibration function
        y_lables:   Physical values/ labels for data points
    Output: 
        max:        Maximum deviation between label and calibrated data point. 
        mean:       Mean deviation between labels and calibrated data points.
        deviation:  Deviation between labels and calibrated data points in Pa.
'''

def max_mean_deviation(y_kalib, y_labels):

    deviation = np.sqrt(np.power((y_kalib - y_labels), 2))

    max = np.amax(deviation)
    mean = np.mean(deviation)

    return max, mean, deviation

In [25]:
# Calculating Maximum NRMSE, corrected R^2 value and maximum deviation for original data without calibration function (90 SPS)
rmse, nrmse = effective_value_of_error(X_90_lin[:, 0], Y, data_points)
r_square, r_square_corr = R_square(X_90_lin[:, 0], Y, data_points, 1)
max, mean, deviation = max_mean_deviation(X_90_lin[:, 0], Y)
print('Original Data:   Max: ' + str(round(max,2)) + ' Rmse: ' + str(round(rmse,2)) + ' R^2: ' + str(r_square_corr))

# Calculating Maximum NRMSE, corrected R^2 value and maximum deviation for linear function (90 SPS)
rmse, nrmse = effective_value_of_error(y_90_lin, Y, data_points)
r_square, r_square_corr = R_square(y_90_lin, Y, data_points, 1)
max, mean, deviation_lin = max_mean_deviation(y_90_lin, Y)
print('Linear:          Max: ' + str(round(max,2)) + ' Rmse: ' + str(round(rmse,2)) + ' R^2: ' + str(r_square_corr))

# Calculating Maximum NRMSE, corrected R^2 value and maximum deviation for linear function <= 1000Pa (90 SPS)
rmse, nrmse = effective_value_of_error(y_90_lin_supersm, Y_small, data_points)
r_square, r_square_corr = R_square(y_90_lin_supersm, Y_small, data_points, 1)
max, mean, deviation_lin_small = max_mean_deviation(y_90_lin_supersm, Y_small)
print('Linear small:    Max: ' + str(round(max,2)) + ' Rmse: ' + str(round(rmse,2)) + ' R^2: ' + str(r_square_corr))

# Calculating Maximum NRMSE, corrected R^2 value and maximum deviation for quadratic function (90 SPS)
rmse, nrmse = effective_value_of_error(y_90_poly2, Y, data_points)
r_square, r_square_corr = R_square(y_90_poly2, Y, data_points, 2)
max, mean, deviation_poly2 = max_mean_deviation(y_90_poly2, Y)
print('Polynom 2.:      Max: ' + str(round(max,2)) + ' Rmse: ' + str(round(rmse,2)) + ' R^2: ' + str(r_square_corr))

# Calculating Maximum NRMSE, corrected R^2 value and maximum deviation for quadratic function <= 1000Pa (90 SPS)
rmse, nrmse = effective_value_of_error(y_90_poly2_supersm, Y_small, data_points)
r_square, r_square_corr = R_square(y_90_poly2_supersm, Y_small, data_points, 2)
max, mean, deviation_poly2_small = max_mean_deviation(y_90_poly2_supersm, Y_small)
print('Polynom 2. small:  Max: ' + str(round(max,2)) + ' Rmse: ' + str(round(rmse,2)) + ' R^2: ' + str(r_square_corr))

# Calculating Maximum NRMSE, corrected R^2 value and maximum deviation for cubic function (90 SPS)
rmse, nrmse = effective_value_of_error(y_90_poly3, Y, data_points)
r_square, r_square_corr = R_square(y_90_poly3, Y, data_points, 3)
max, mean, deviaiton_poly3 = max_mean_deviation(y_90_poly3, Y)
print('Polynom 3.:      Max: ' + str(round(max,2)) + ' Rmse: ' + str(round(rmse,2)) + ' R^2: ' + str(r_square_corr))

# Calculating Maximum NRMSE, corrected R^2 value and maximum deviation for 4th order function (90 SPS)
rmse, nrmse = effective_value_of_error(y_90_poly4, Y, data_points)
r_square, r_square_corr = R_square(y_90_poly4, Y, data_points, 4)
max, mean, deviation_poly4 = max_mean_deviation(y_90_poly4, Y)
print('Polynom 4.:      Max: ' + str(round(max,2)) + ' Rmse: ' + str(round(rmse,2)) + ' R^2: ' + str(r_square_corr))

Original Data:   Max: 24.0 Rmse: 107.45 R^2: 0.9985976442185722
Linear:          Max: 1.65 Rmse: 3.08 R^2: 0.9999999492359124
Linear small:    Max: 1.47 Rmse: 2.06 R^2: 0.9999994482205226
Polynom 2.:      Max: 1.64 Rmse: 3.02 R^2: 0.9999999511317619
Polynom 2. small:  Max: 1.45 Rmse: 2.06 R^2: 0.9999994482504194
Polynom 3.:      Max: 1.65 Rmse: 3.12 R^2: 0.9999999512677793
Polynom 4.:      Max: 141.28 Rmse: 236.14 R^2: 0.9998151235884678


In [22]:
# Calculating Maximum NRMSE, corrected R^2 value and maximum deviation for 20 SPS
rmse, nrmse = effective_value_of_error(y_20_1, Y_20, data_points_20)
r_square, r_square_corr = R_square(y_20_1, Y_20, data_points_20, 1)
max, mean, deviation_20 = max_mean_deviation(y_20_1, Y_20)
max_org, mean_org, deviation_org = max_mean_deviation(X_20_1[:, 0], Y_20)
print('Max before 20: ' + str(round(max_org,2)), 'Max calibrated 20: ' + str(round(max,2)), 'Mean before 20: ' + str(round(mean_org,2)), 'Mean calibrated 20: ' + str(round(mean,2)), ' Nrmse 20: ' + str(round(nrmse,2)) + ' R^2_corr 20: ' + str(round(r_square_corr,2)))

# Calculating Maximum NRMSE, corrected R^2 value and maximum deviation for 90 SPS
rmse, nrmse = effective_value_of_error(y_90_1, Y_90, data_points_90)
r_square, r_square_corr = R_square(y_90_1, Y_90, data_points_90, 1)
max, mean, deviation_90 = max_mean_deviation(y_90_1, Y_90)
max_org, mean_org, deviation_org = max_mean_deviation(X_90_1[:, 0], Y_90)
print('Max before 90: ' + str(round(max_org,2)), 'Max calibrated 90: ' + str(round(max,2)), 'Mean before 90: ' + str(round(mean_org,2)), 'Mean calibrated 90: ' + str(round(mean,2)),' Nrmse 90: ' + str(round(nrmse,2)) + ' R^2_corr 90: ' + str(round(r_square_corr,2)))

# Calculating Maximum NRMSE, corrected R^2 value and maximum deviation for 175 SPS
rmse, nrmse = effective_value_of_error(y_175_1, Y_175, data_points_175)
r_square, r_square_corr = R_square(y_175_1, Y_175, data_points_175, 1)
max, mean, deviation_175 = max_mean_deviation(y_175_1, Y_175)
max_org, mean_org, deviation_org = max_mean_deviation(X_175_1[:, 0], Y_175)
print('Max before 175: ' + str(round(max_org,2)), 'Max calibrated 175: ' + str(round(max,2)), 'Mean before 175: ' + str(round(mean_org,2)), 'Mean calibrated 175: ' + str(round(mean,2)), ' Nrmse 175: ' + str(round(nrmse,2)) + ' R^2_corr 175: ' + str(round(r_square_corr,2)))

# Calculating Maximum NRMSE, corrected R^2 value and maximum deviation for 330 SPS
rmse, nrmse = effective_value_of_error(y_330_1, Y_330, data_points_330)
r_square, r_square_corr = R_square(y_330_1, Y_330, data_points_330, 1)
max, mean, deviation_330 = max_mean_deviation(y_330_1, Y_330)
max_org, mean_org, deviation_org = max_mean_deviation(X_330_1[:, 0], Y_330)
print('Max before 330: ' + str(round(max_org,2)), 'Max calibrated 330: ' + str(round(max,2)), 'Mean before 330: ' + str(round(mean_org,2)), 'Mean calibrated 330: ' + str(round(mean,2)), ' Nrmse 330: ' + str(round(nrmse,2)) + ' R^2_corr 330: ' + str(round(r_square_corr,2)))

# Calculating Maximum NRMSE, corrected R^2 value and maximum deviation for 1000 SPS
rmse, nrmse = effective_value_of_error(y_1000_1, Y_1000, data_points_1000)
r_square, r_square_corr = R_square(y_1000_1, Y_1000, data_points_1000, 1)
max, mean, deviation_20 = max_mean_deviation(y_1000_1, Y_1000)
max_org, mean_org, deviation_org = max_mean_deviation(X_1000_1[:, 0], Y_1000)
print('Max before 1000: ' + str(round(max_org,2)), 'Max calibrated 1000: ' + str(round(max,2)), 'Mean before 1000: ' + str(round(mean_org,2)), 'Mean calibrated 1000: ' + str(round(mean,2)), ' Nrmse 1000: ' + str(round(nrmse,2)) + ' R^2_corr 1000: ' + str(round(r_square_corr,2)))

Max before 20: 24.0 Max calibrated 20: 1.46 Mean before 20: 19.14 Mean calibrated 20: 0.34  Nrmse 20: 0.01 R^2_corr 20: 1.0
Max before 90: 24.0 Max calibrated 90: 1.59 Mean before 90: 19.27 Mean calibrated 90: 0.39  Nrmse 90: 0.02 R^2_corr 90: 1.0
Max before 175: 1986.0 Max calibrated 175: 2144.79 Mean before 175: 126.69 Mean calibrated 175: 55.57  Nrmse 175: 2.6 R^2_corr 175: 1.0
Max before 330: 24.0 Max calibrated 330: 1.97 Mean before 330: 18.97 Mean calibrated 330: 0.46  Nrmse 330: 0.02 R^2_corr 330: 1.0
Max before 1000: 25.0 Max calibrated 1000: 10.06 Mean before 1000: 18.72 Mean calibrated 1000: 0.72  Nrmse 1000: 0.04 R^2_corr 1000: 1.0


# Manuelle Kalibrierung

In [30]:
'''
    Calculating p_raw and calling the function which calculates the calibrated values. 
    Input: 
        X_val:          Uncalibrated data points.
        T_cel:          Temperature values in Celcius
        num_of_sensor:  Number of sensor (Sensor 1-5)  
    Output: 
        P_comp:         Temperature compensated and pressure calibrated values  
'''

# Calibration Coefficants for Sensor 1 to 3 (read from EEPROM)
P_range = 120
P_min = -60
Offset_0_S1 = 588775.38
Offset_1_S1 = -20.39
Offset_2 = 0
Offset_3 = 0
Offset_0_S2 = 212262.17
Offset_1_S2 = -24.39
Offset_0_S3 = 637576.38
Offset_1_S3 = -25.31

Span_0_S1 = -2906524
Span_1_S1 = 185.29
Span_2 = -0.01
Span_3 = 0
Span_0_S2 = -2886532.5
Span_1_S2 = 184.10
Span_0_S3 = -2951100
Span_1_S3 = 187.55

Shape_0 = 0.5
Shape_1 = 1
Shape_2 = 0.01
Shape_3 = 0

def calc_p_raw(X_val, T_Cel, num_of_sensor):

    # converting measured values from Pa to mbar 
    X_val = X_val / 100

    # Equation 4
    P_comp_FS = (X_val - P_min) / P_range
    # Equation 3 
    p = Shape_1 / Shape_2
    q = (Shape_0 - P_comp_FS) / Shape_2
    P_int2 = - (p / 2) + np.sqrt(np.power((p / 2), 2) - q)
    # P_int2 = - (p / 2) - np.sqrt(np.power((p / 2), 2) - q)
    if(num_of_sensor == 1):
        # Equation 2
        P_int1 = P_int2 * Span_0_S1
        # Equation 1
        P_raw = P_int1 + Offset_0_S1
    elif(num_of_sensor == 2):
        # Equation 2
        P_int1 = P_int2 * Span_0_S2
        # Equation 1
        P_raw = P_int1 + Offset_0_S2
    elif(num_of_sensor == 3):
        # Equation 2
        P_int1 = P_int2 * Span_0_S3
        # Equation 1
        P_raw = P_int1 + Offset_0_S3
    else: 
        print('Sensor not defined.')

    P_comp = compensate_calibrate(P_raw, T_Cel, num_of_sensor)

    return P_comp

In [29]:
'''
    Calculates compensated/ calibrated values. Coefficiants are read from the EEPROM of the sensors. 
    Input: 
        P_raw:          Raw pressure values
        T_cel:          Temperature values in Celcius
        num_of_sensor:  Number of sensor (Sensor 1-5 
    Output: 
        P_comp:         Temperature compensated and pressure calibrated values
'''

def compensate_calibrate(P_raw, T_Cel, num_of_sensor):
    T_Cel = T_Cel / 100
    T_Cel = T_Cel / 0.03125 

    if(num_of_sensor == 1):
        # Equation 1
        P_int1 = P_raw - (Offset_3 * np.power(T_Cel, 3) + Offset_2 * np.power(T_Cel, 2) + Offset_1_S1 * T_Cel + Offset_0_S1) 
        # Equation 2
        P_int2 = P_int1 / (Span_3 * np.power(T_Cel, 3) + Span_2 * np.power(T_Cel, 2) + Span_1_S1 * T_Cel + Span_0_S1)
    elif(num_of_sensor == 2):
        # Equation 1
        P_int1 = P_raw - (Offset_3 * np.power(T_Cel, 3) + Offset_2 * np.power(T_Cel, 2) + Offset_1_S2 * T_Cel + Offset_0_S2) 
        # Equation 2
        P_int2 = P_int1 / (Span_3 * np.power(T_Cel, 3) + Span_2 * np.power(T_Cel, 2) + Span_1_S2 * T_Cel + Span_0_S2)
    elif(num_of_sensor == 3):
        # Equation 1
        P_int1 = P_raw - (Offset_3 * np.power(T_Cel, 3) + Offset_2 * np.power(T_Cel, 2) + Offset_1_S3 * T_Cel + Offset_0_S3) 
        # Equation 2
        P_int2 = P_int1 / (Span_3 * np.power(T_Cel, 3) + Span_2 * np.power(T_Cel, 2) + Span_1_S3 * T_Cel + Span_0_S3)
    else: 
        print('Sensor ungültig.')
        
    # Equation 3
    P_comp_FS = Shape_3 * np.power(P_int2, 3) + Shape_2 * np.power(P_int2, 2) + Shape_1 * P_int2 + Shape_0
    # Equation 4
    P_comp = (P_comp_FS * P_range) + P_min
    
    return P_comp

# Auswertung Flugversuchsdaten

In [33]:
# Loads data from a CSV file.

def load_inflight_data(fname, num_of_col, len_header, data_type, num_data_points, delimiter):
    
    # load the data
    data = np.genfromtxt(fname, dtype=None, delimiter=delimiter, usecols= num_of_col) 
    
    return data

In [28]:
''' Calibrates in-flight measured pressure values with calucalted calibration funciton.
    Input: 
        data:       Uncalibrated data points.
        W:          Contains values of calibration function 
    Output: 
        Y:          Calibrated data points.        
'''

def calib_inflight_data(data, W): 
    
    Y = data * W[0] + (W[1] / 100)
    
    return Y

In [ ]:
'''
    Reading data from flight test.
'''

DLR_file = '/Users/leaschwalb/Documents/UNI/1_Bachelorarbeit/Messungen/Flugversuch/3ter/Zusammenfassung_werte.txt'

# number of columns in data set
ESA_Sensor1 = 60
ESA_Sensor2 = 61
ESA_Sensor3 = 62
ESA_Sensor1_Temp = 67
ESA_Sensor2_Temp = 68 
ESA_Sensor3_Temp = 69

DLR_Höhe = 7
DLR_Diff_a = 31
DLR_Diff_b = 32
DLR_Stau = 29
DLR_Statik = 30
DLR_a = 33
DLR_b = 34
DLR_Temp_C = 35
Sek_since_Start = 74

cols_to_read_DLR = (DLR_Höhe, DLR_Diff_a, DLR_Diff_b, DLR_a, DLR_Diff_b, DLR_Stau, DLR_Statik, DLR_Temp_C, Sek_since_Start)
cols_to_read_ESA_S1 = (ESA_Sensor1, ESA_Sensor1_Temp)
cols_to_read_ESA_S2 = (ESA_Sensor2, ESA_Sensor2_Temp)
cols_to_read_ESA_S3 = (ESA_Sensor3, ESA_Sensor3_Temp)

# Loading data from file
DLR_data = load_inflight_data(DLR_file, cols_to_read_DLR, 85, float, 43950, " ")
ESA_data_S1 = load_inflight_data(DLR_file, cols_to_read_ESA_S1, 85, int, 43950, " ")
ESA_data_S2 = load_inflight_data(DLR_file, cols_to_read_ESA_S2, 85, int, 43950, " ")
ESA_data_S3 = load_inflight_data(DLR_file, cols_to_read_ESA_S3, 85, int, 43950, " ")

# Calculating pressure temperature-compensated and pressure-calibrated sensor data 
Sensor_1_comp = calc_p_raw(ESA_data_S1[:, 0], ESA_data_S1[:, 1], 1)
Sensor_2_comp = calc_p_raw(ESA_data_S2[:, 0], ESA_data_S2[:, 1], 2)
Sensor_3_comp = calc_p_raw(ESA_data_S3[:, 0], ESA_data_S3[:, 1], 3)

# Applying linear calibration function 
Sensor_1_calib = calib_inflight_data(Sensor_1_comp, W_90_lin_1)
Sensor_2_calib = calib_inflight_data(Sensor_2_comp, W_90_lin_2)
Sensor_3_calib = calib_inflight_data(Sensor_3_comp, W_90_lin_3)

Sensor_2_calib_invert = Sensor_2_calib * -1

# Subtracting calibration offset 
DLR_data[:, 5] = DLR_data[:, 5] - 0.117
DLR_data[:, 1] = DLR_data[:, 1] - 0.144
DLR_data[:, 2] = DLR_data[:, 2] - 0.138